## Install Dependencies

In [2]:
!pip install tqdm
!pip install torch
!pip install torchvision
!pip install kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

## Load DataSet

In [3]:
import os
os.environ['KAGGLE_USERNAME'] = "manishjoshimj"
os.environ['KAGGLE_KEY'] = "14910efaaa9fb2b9f03bd5f3dc575116"

!kaggle datasets download -d manishjoshimj/organico-and-recyclabler-dataset

Dataset URL: https://www.kaggle.com/datasets/manishjoshimj/organico-and-recyclabler-dataset
License(s): unknown
100% 1.65G/1.66G [00:11<00:00, 85.4MB/s]
100% 1.66G/1.66G [00:11<00:00, 151MB/s] 


In [4]:
# Unzip the dataset

!unzip -q organico-and-recyclabler-dataset.zip -d waste-dataset

## Necessary Imports

In [9]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

In [10]:
# Constants
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 10
DATASET_DIR = '/content/waste-dataset/O_R_WASTE_CLASSIFICATION_DATASET'

In [11]:
# Transforming the dataset using randomization

train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [12]:
# Loading Dataset

train_dataset = datasets.ImageFolder(os.path.join(DATASET_DIR, "TRAIN"), transform=train_transform)
test_dataset = datasets.ImageFolder(os.path.join(DATASET_DIR, "TEST"), transform=test_transform)

In [13]:
# Data Loaders

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2,
    pin_memory=True,
    persistent_workers=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    pin_memory=True,
    persistent_workers=True
)

## Model Training

In [14]:
# DenseNet201 Refining

def get_model():
    model = models.densenet201(pretrained=True)

    # Freeze all pretrained layers
    for param in model.parameters():
        param.requires_grad = False

    # Replace the classifier
    num_ftrs = model.classifier.in_features
    model.classifier = nn.Sequential(
        nn.Linear(num_ftrs, 256),
        nn.ReLU(),
        nn.Dropout(0.4),
        nn.Linear(256, 2)  # 2 classes: O (organic), R (recyclable)
    )

    return model.to(device)

model = get_model()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet201-c1103571.pth" to /root/.cache/torch/hub/checkpoints/densenet201-c1103571.pth
100%|██████████| 77.4M/77.4M [00:00<00:00, 126MB/s]


In [15]:
# Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=1e-4)

In [16]:
def train_model(model, train_loader, test_loader, epochs, criterion, optimizer, device):
    train_acc, val_acc = [], []

    for epoch in range(epochs):
        model.train()
        correct = total = 0
        running_loss = 0.0

        print(f"\nEpoch {epoch + 1}/{epochs}")
        train_bar = tqdm(train_loader, desc="Training", leave=False)

        for images, labels in train_bar:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            preds = torch.argmax(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            running_loss += loss.item()

            train_bar.set_postfix({
                'Loss': f'{running_loss / total:.4f}',
                'Acc': f'{correct / total:.4f}'
            })

        acc = correct / total
        train_acc.append(acc)

        # Validation phase
        model.eval()
        correct = total = 0
        val_bar = tqdm(test_loader, desc="Validating", leave=False)

        with torch.no_grad():
            for images, labels in val_bar:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                preds = torch.argmax(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

        val_accuracy = correct / total
        val_acc.append(val_accuracy)

        print(f"Epoch {epoch+1} Completed - Train Acc: {train_acc[-1]:.4f} | Val Acc: {val_accuracy:.4f}")

    return model, train_acc, val_acc


In [17]:
# Start Training
model, train_acc, val_acc = train_model(model, train_loader, test_loader, EPOCHS, criterion, optimizer, device)


Epoch 1/10


Epoch 1 Completed - Train Acc: 0.9403 | Val Acc: 0.9522

Epoch 2/10


Epoch 2 Completed - Train Acc: 0.9555 | Val Acc: 0.9464

Epoch 3/10


Epoch 3 Completed - Train Acc: 0.9599 | Val Acc: 0.9480

Epoch 4/10


Epoch 4 Completed - Train Acc: 0.9621 | Val Acc: 0.9555

Epoch 5/10


...

Epoch 6 Completed - Train Acc: 0.9689 | Val Acc: 0.9491

Epoch 7/10


Epoch 7 Completed - Train Acc: 0.9703 | Val Acc: 0.9434

Epoch 8/10


Epoch 8 Completed - Train Acc: 0.9718 | Val Acc: 0.9398

Epoch 9/10


Epoch 9 Completed - Train Acc: 0.9737 | Val Acc: 0.9486

Epoch 10/10


Epoch 10 Completed - Train Acc: 0.9751 | Val Acc: 0.9524


In [18]:
# Save final model checkpoint
torch.save(model.state_dict(), 'densenet201_final_checkpoint.pth')
print("Model saved to densenet201_final_checkpoint.pth")

Model saved to densenet201_final_checkpoint.pth


In [19]:
# Download the model
from google.colab import files #noqa
files.download('densenet201_final_checkpoint.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>